## Reject

There are several strategies for handling concurrent runs in your graph. This notebook covers how to use the `reject` option - please see the other how-to guides in the "Double Texting" directory to learn about the other methods.

First, let's import our required packages and instantiate our client, assistant, and thread.

In [ ]:
import httpx
from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client

In [ ]:
client = get_client()

In [ ]:
assistant = await client.assistants.create("agent")

In [ ]:
thread = await client.threads.create()

In [ ]:
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "what's the weather in sf?"}]},
)

In [ ]:
try:
    await client.runs.create(
        thread["thread_id"],
        assistant["assistant_id"],
        input={
            "messages": [{"role": "human", "content": "what's the weather in nyc?"}]
        },
        multitask_strategy="reject",
    )
except httpx.HTTPStatusError as e:
    print("Failed to start concurrent run", e)

Failed to start concurrent run Client error '409 Conflict' for url 'http://localhost:8123/threads/f9e7088b-8028-4e5c-88d2-9cc9a2870e50/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409


We can verify that the original thread finished executing:

In [ ]:
# wait until the original run completes
await client.runs.join(thread["thread_id"], run["run_id"])

In [ ]:
state = await client.threads.get_state(thread["thread_id"])

In [ ]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

what's the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_01CyewEifV2Kmi7EFKHbMDr1', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01CyewEifV2Kmi7EFKHbMDr1)
 Call ID: toolu_01CyewEifV2Kmi7EFKHbMDr1
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.accuweather.com/en/us/san-francisco/94103/june-weather/347629", "content": "Get the monthly weather forecast for San Francisco, CA, including daily high/low, historical averages, to help you plan ahead."}]
================================== Ai Message ==================================

According to the search results from Tavily, the current weather in San Franc